# data load
- {종목명 : 데이터프레임 , ....}

In [37]:
import pandas as pd
import glob
import os

def load_optimized_crypto_data(data_path):
    """2차 라벨링된 암호화폐 CSV 파일들을 딕셔너리로 로드"""
    
    # 최적화된 파일들 찾기
    csv_files = glob.glob(os.path.join(data_path, "*_optimized.csv"))
    
    if not csv_files:
        print(f"❌ {data_path}에서 *_optimized.csv 파일을 찾을 수 없습니다.")
        return {}
    
    processed_stocks = {}
    
    print(f"📁 {len(csv_files)}개 최적화 파일 로딩 중...")
    
    for file_path in csv_files:
        try:
            # 심볼명 추출
            filename = os.path.basename(file_path)
            symbol = filename.replace('_optimized.csv', '')
            
            # CSV 읽기
            df = pd.read_csv(file_path)
            
            # Date 컬럼 처리
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'])
                df.set_index('Date', inplace=True)
            
            processed_stocks[symbol] = df
            print(f"✅ {symbol}: {df.shape}")
            
        except Exception as e:
            print(f"❌ {filename}: {e}")
    
    print(f"🎯 로딩 완료: {len(processed_stocks)}개 종목")
    return processed_stocks

# 사용법
data_path = "/workspace/AI모델/projects/coin/data/v01/optimized"
processed_stocks = load_optimized_crypto_data(data_path)

📁 50개 최적화 파일 로딩 중...
✅ AAVE: (1821, 58)
✅ ADA: (2879, 58)
✅ AETHWETH: (138, 58)
✅ ALGO: (2290, 58)
✅ AVAX: (1833, 58)
✅ BCH: (2879, 58)
✅ BGB: (1521, 58)
✅ BNB: (2879, 58)
✅ BTCB: (2293, 58)
✅ BTC: (3922, 58)
✅ CBBTC32994: (380, 58)
✅ CRO: (2479, 58)
✅ DAI: (2136, 58)
✅ DOGE: (2879, 58)
✅ DOT: (1864, 58)
✅ ENA: (543, 58)
✅ ETH: (2879, 58)
✅ HBAR: (2202, 58)
✅ HYPE32196: (302, 58)
✅ JITOSOL: (1058, 58)
✅ LEO: (2321, 58)
✅ LINK: (2879, 58)
✅ LTC: (3922, 58)
✅ MNT27075: (800, 58)
✅ NEAR: (1809, 58)
✅ OKB: (2342, 58)
✅ PEPE24478: (894, 58)
✅ SEI: (771, 58)
✅ SHIB: (1842, 58)
✅ SOL: (1996, 58)
✅ STETH: (1739, 58)
✅ SUI20947: (878, 58)
✅ SUSDE: (583, 58)
✅ TAO22974: (937, 58)
✅ TON11419: (1492, 58)
✅ TRX: (2879, 58)
✅ UNI7083: (1835, 58)
✅ USDC: (2546, 58)
✅ USDE29470: (585, 58)
✅ USDS33039: (373, 58)
✅ USDT: (2879, 58)
✅ WBETH: (884, 58)
✅ WBTC: (2432, 58)
✅ WEETH: (654, 58)
✅ WETH: (2812, 58)
✅ WSTETH: (1451, 58)
✅ WTRX: (1302, 58)
✅ XLM: (2879, 58)
✅ XMR: (2879, 58)
✅ XRP: (2879, 58)
🎯 로딩

# 피처엔지니어링 정보
- Date 제외 : XGBoost는 트리 기반 모델이라 날짜를 숫자로 변환해도 의미있는 패턴을 찾기 어려움
- 백테스팅할때만 있으면됨.

In [38]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, RobustScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

# 학습에서 제외할 컬럼들
EXCLUDE_FROM_TRAINING = [
    'Date', 'Symbol', 'Future_Label', 'Label', 'Optimized_Label', 'Volatility_Signal', 
    'Volume_Breakout_Signal', 'Composite_Score',
    'Label_Name', 'Optimized_Label_Name', 'Future_Label_Name',
    'Open', 'High', 'Low', 'Close', 'Volume',  # 원본 가격/거래량
    'Adj Close', 'Dividend', 'Stock Split',  # 기타 원본 데이터
]

# 백테스팅용 보존 컬럼들
BACKTEST_PRESERVE_COLUMNS = [
    'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 
    'Label', 'Label_Name', 'Optimized_Label', 'Optimized_Label_Name', 'Symbol'
]

# 정규화에서 제외할 Signal/Binary 변수들
SIGNAL_FEATURES = [
    # 이진 시그널들 (0/1)
    'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end',
    # 3단계 시그널들 (0/1/2)
    'Formula3_Signal', 'Momentum_Signal',
    # 기타 이진 지표들
    'high_volatility_regime',
    # RSI 관련 이진 지표들
    'RSI_7_overbought', 'RSI_7_oversold', 'RSI_14_overbought', 'RSI_14_oversold',
    'RSI_30_overbought', 'RSI_30_oversold',
    # Stochastic 관련 이진 지표들
    'Stoch_1_K_above_D', 'Stoch_3_K_above_D', 'Stoch_4_K_above_D', 'Stoch_5_K_above_D',
    'Stoch_6_K_above_D', 'Stoch_14_K_above_D', 'Stoch_1_overbought', 'Stoch_3_overbought',
    'Stoch_4_overbought', 'Stoch_5_overbought', 'Stoch_6_overbought', 'Stoch_14_overbought',
    'Stoch_1_oversold', 'Stoch_3_oversold', 'Stoch_4_oversold', 'Stoch_5_oversold',
    'Stoch_6_oversold', 'Stoch_14_oversold',
    # CCI 관련 이진 지표들
    'CCI_3_overbought', 'CCI_3_oversold', 'CCI_4_overbought', 'CCI_4_oversold',
    'CCI_7_overbought', 'CCI_7_oversold', 'CCI_14_overbought', 'CCI_14_oversold',
    # SMI 관련 이진 지표들
    'SMI_overbought_40', 'SMI_oversold_40', 'SMI_overbought_50', 'SMI_oversold_50',
    # RSI 다이버전스 이진 지표들
    'RSI_7_bullish_divergence', 'RSI_7_bearish_divergence', 
    'RSI_14_bullish_divergence', 'RSI_14_bearish_divergence',
    # MACD 관련 이진 지표들
    'MACD_7_14_above_signal', 'MACD_14_30_above_signal',
    # 원형 인코딩 (이미 정규화됨)
    'month_sin', 'month_cos', 'dow_sin', 'dow_cos'
]


# 전처리 및 정규화

In [39]:
# =============================================================================
# CryptoXGBoostPreprocessor 클래스 (70:15:15 단순 분할)
# =============================================================================

class CryptoXGBoostPreprocessor:
    """암호화폐 XGBoost 전처리기 - 종목별 정규화"""
    
    def __init__(self, forecast_horizon=1):
        self.forecast_horizon = forecast_horizon
        self.symbol_scalers = {}  # 종목별 스케일러 저장
        
    def prepare_single_crypto(self, df, symbol):
        """개별 암호화폐 전처리 및 종목별 정규화"""
        df_processed = df.copy()
        
        print(f"  처리 중: {symbol} ({df.shape})")
        
        # Date 정보 보존 (인덱스에서 컬럼으로)
        if 'Date' not in df_processed.columns and isinstance(df_processed.index, pd.DatetimeIndex):
            df_processed['Date'] = df_processed.index
        
        # 1. Future_Label 생성
        if 'Future_Label' not in df_processed.columns:
            if 'Optimized_Label' in df_processed.columns:
                df_processed['Future_Label'] = df_processed['Optimized_Label'].shift(-self.forecast_horizon)
            else:
                print(f"    Warning: {symbol}에 Optimized_Label 없음")
                return None
        
        # 2. 심볼 정보 추가
        df_processed['Symbol'] = symbol
        
        # 3. 무한값 및 극값 처리
        numeric_cols = df_processed.select_dtypes(include=[np.number]).columns
        df_processed[numeric_cols] = df_processed[numeric_cols].replace([np.inf, -np.inf], np.nan)
        
        # 극값 클리핑 (종목별로)
        for col in numeric_cols:
            if df_processed[col].notna().sum() > 0:
                p99 = df_processed[col].quantile(0.999)
                p01 = df_processed[col].quantile(0.001)
                if not pd.isna(p99) and not pd.isna(p01):
                    df_processed[col] = df_processed[col].clip(lower=p01, upper=p99)
        
        # 4. NaN 채우기
        df_processed[numeric_cols] = df_processed[numeric_cols].fillna(0)
        
        # 5. 종목별 정규화 적용
        df_normalized = self.apply_symbol_scaling(df_processed, symbol)
        
        # 6. Future_Label이 있는 데이터만 유지
        df_final = df_normalized.dropna(subset=['Future_Label']).copy()
        
        print(f"    완료: {df_final.shape}, 라벨수={df_final['Future_Label'].nunique()}")
        
        return df_final
    
    def apply_symbol_scaling(self, df, symbol):
        """종목별 스케일링 적용"""
        df_scaled = df.copy()
        
        # 스케일링 제외 컬럼들
        exclude_cols = set(EXCLUDE_FROM_TRAINING + SIGNAL_FEATURES + ['year', 'month', 'quarter', 'day_of_week'])
        string_cols = set(df_scaled.select_dtypes(include=['object']).columns)
        all_exclude = exclude_cols | string_cols
        
        # 스케일링 대상 피쳐
        scale_features = [col for col in df_scaled.columns if col not in all_exclude]
        
        if scale_features:
            # 종목별 스케일러 생성/사용
            if symbol not in self.symbol_scalers:
                self.symbol_scalers[symbol] = StandardScaler()
            
            try:
                df_scaled[scale_features] = self.symbol_scalers[symbol].fit_transform(df_scaled[scale_features])
            except Exception as e:
                print(f"    스케일링 실패 ({symbol}): {e}")
                df_scaled[scale_features] = 0
        
        return df_scaled
    
    def split_symbol_data(self, df, train_ratio=0.7, val_ratio=0.15):
        """종목별 시계열 데이터 분할 - 70:15:15"""
        n = len(df)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))
        
        train_data = df.iloc[:train_end].copy()
        val_data = df.iloc[train_end:val_end].copy()
        test_data = df.iloc[val_end:].copy()
        
        return train_data, val_data, test_data
    
    def process_all_cryptos(self, crypto_data, output_dir):
        """전체 암호화폐 처리 - 종목별 분할"""
        print("암호화폐 XGBoost 전처리 시작 (종목별 정규화)")
        print("=" * 60)
        
        # 결과 저장용
        all_train_data = []
        all_val_data = []
        all_test_data = []
        backtest_data = {}  # 백테스팅용 원본 데이터
        symbol_stats = {}
        
        for symbol, df in crypto_data.items():
            try:
                # 1. 백테스팅용 원본 데이터 보존 (전체 기간)
                df_reset = df.reset_index()
                if 'Date' not in df_reset.columns and isinstance(df.index, pd.DatetimeIndex):
                    df_reset['Date'] = df.index
                
                available_cols = [col for col in BACKTEST_PRESERVE_COLUMNS if col in df_reset.columns]
                backtest_data[symbol] = df_reset[available_cols].copy()
                
                # 2. 전처리 및 종목별 정규화
                processed_df = self.prepare_single_crypto(df, symbol)
                
                if processed_df is None or len(processed_df) < 30:
                    print(f"    스킵: {symbol} (데이터 부족)")
                    continue
                
                # 3. 종목별 시계열 분할 (70:15:15)
                train_data, val_data, test_data = self.split_symbol_data(processed_df)
                
                # 4. 각 세트에 데이터 추가
                if len(train_data) > 0:
                    all_train_data.append(train_data)
                if len(val_data) > 0:
                    all_val_data.append(val_data)
                if len(test_data) > 0:
                    all_test_data.append(test_data)
                
                # 5. 종목별 통계 저장
                symbol_stats[symbol] = {
                    'total_rows': len(processed_df),
                    'train_rows': len(train_data),
                    'val_rows': len(val_data),
                    'test_rows': len(test_data),
                    'label_dist': processed_df['Future_Label'].value_counts().to_dict()
                }
                
            except Exception as e:
                print(f"    오류: {symbol} - {e}")
                continue
        
        # 6. 데이터 결합
        if not all_train_data:
            raise ValueError("처리된 데이터가 없습니다!")
        
        train_df = pd.concat(all_train_data, axis=0, ignore_index=True)
        val_df = pd.concat(all_val_data, axis=0, ignore_index=True) if all_val_data else pd.DataFrame()
        test_df = pd.concat(all_test_data, axis=0, ignore_index=True) if all_test_data else pd.DataFrame()
        
        print(f"\n종목별 분할 후 전체 데이터:")
        print(f"  훈련: {len(train_df):,}개 (70%)")
        print(f"  검증: {len(val_df):,}개 (15%)") 
        print(f"  테스트: {len(test_df):,}개 (15%)")
        
        # 7. 종목별 통계 출력
        print(f"\n종목별 데이터 분포:")
        for symbol, stats in symbol_stats.items():
            print(f"  {symbol}: train={stats['train_rows']}, val={stats['val_rows']}, test={stats['test_rows']}")
        
        # 8. 전체 라벨 분포
        if len(train_df) > 0:
            print(f"\n전체 라벨 분포:")
            for dataset_name, dataset in [('Train', train_df), ('Val', val_df), ('Test', test_df)]:
                if len(dataset) > 0:
                    label_counts = dataset['Future_Label'].value_counts().sort_index()
                    print(f"  {dataset_name}:")
                    for label, count in label_counts.items():
                        label_name = {0: 'Sell', 1: 'Hold', 2: 'Buy'}[int(label)]
                        print(f"    {int(label)} ({label_name}): {count:,}개")
        
        # 9. 학습용 피쳐 선택
        training_features = [col for col in train_df.columns if col not in EXCLUDE_FROM_TRAINING]
        print(f"\n학습용 피쳐: {len(training_features)}개")
        
        # 10. 데이터 저장
        self.save_processed_data(train_df, val_df, test_df, backtest_data, 
                               training_features, symbol_stats, output_dir)
        
        return train_df, val_df, test_df, backtest_data, training_features
    
    def save_processed_data(self, train_df, val_df, test_df, backtest_data, 
                          training_features, symbol_stats, output_dir):
        """처리된 데이터 저장"""
        os.makedirs(output_dir, exist_ok=True)
        
        # 학습용 데이터 저장
        train_df.to_csv(os.path.join(output_dir, 'train_data.csv'), index=False)
        if len(val_df) > 0:
            val_df.to_csv(os.path.join(output_dir, 'val_data.csv'), index=False)
        if len(test_df) > 0:
            test_df.to_csv(os.path.join(output_dir, 'test_data.csv'), index=False)
        
        # 전체 학습 데이터 (train + val + test)
        all_data = pd.concat([train_df, val_df, test_df], axis=0, ignore_index=True)
        all_data.to_csv(os.path.join(output_dir, 'xgboost_training_data.csv'), index=False)
        
        # 백테스팅 원본 데이터 저장
        backtest_dir = os.path.join(output_dir, 'backtest_data')
        os.makedirs(backtest_dir, exist_ok=True)
        for symbol, data in backtest_data.items():
            data.to_csv(os.path.join(backtest_dir, f'{symbol}_backtest.csv'), index=False)
        
        # 전처리기 및 메타데이터
        joblib.dump(self, os.path.join(output_dir, 'crypto_xgboost_preprocessor.pkl'))
        
        with open(os.path.join(output_dir, 'training_features.txt'), 'w') as f:
            for feature in training_features:
                f.write(f"{feature}\n")
        
        # 종목별 통계
        import json
        with open(os.path.join(output_dir, 'symbol_stats.json'), 'w') as f:
            json.dump(symbol_stats, f, indent=2)
        
        print(f"\n데이터 분할 완료:")
        print(f"   훈련: 70%")
        print(f"   검증: 15%") 
        print(f"   테스트: 15%")
        print(f"\n저장 완료: {output_dir}")


# =============================================================================
# prepare_xgboost_data 함수 (70:15:15 단순 분할)
# =============================================================================

def prepare_xgboost_data(processed_stocks, output_dir='./data/xgboost', 
                        forecast_horizon=1, target_symbols=None):
    """
    암호화폐 XGBoost 데이터 전처리 메인 함수 (70:15:15 분할)
    """
    
    print(f"암호화폐 XGBoost 전처리 시작")
    print(f"예측 기간: {forecast_horizon}일")
    print(f"종목별 정규화 적용")
    
    # 타겟 심볼 필터링
    if target_symbols is not None:
        filtered_stocks = {}
        for target in target_symbols:
            clean_target = target.replace('-USD', '').replace('_USDT', '')
            for symbol in processed_stocks.keys():
                clean_symbol = symbol.replace('-USD', '').replace('_USDT', '')
                if clean_symbol == clean_target:
                    filtered_stocks[symbol] = processed_stocks[symbol]
                    break
        processed_stocks = filtered_stocks
        print(f"필터링된 종목: {list(processed_stocks.keys())}")
    
    if not processed_stocks:
        raise ValueError("선택된 종목이 없습니다!")
    
    # 전처리기 실행
    preprocessor = CryptoXGBoostPreprocessor(forecast_horizon=forecast_horizon)
    train_df, val_df, test_df, backtest_data, training_features = preprocessor.process_all_cryptos(
        processed_stocks, output_dir
    )
    
    print("모든 전처리 완료!")
    
    return {
        'train': train_df,
        'val': val_df, 
        'test': test_df,
        'backtest_data': backtest_data,
        'training_features': training_features,
        'preprocessor': preprocessor
    }

In [40]:
output_dir = "/workspace/AI모델/projects/coin/data/v01/crypto_xgboost"

In [41]:
# =============================================================================
# 3. 실행 부분 (기존과 동일)
# =============================================================================

if __name__ == "__main__":
    # 1. 데이터 로드
    data_path = "/workspace/AI모델/projects/coin/data/v01/optimized"
    processed_stocks = load_optimized_crypto_data(data_path)
    
    # 2. 전처리
    result = prepare_xgboost_data(
        processed_stocks=processed_stocks,
        output_dir=output_dir,
        forecast_horizon=1,
        target_symbols=None  # 전체 종목
    )
    
    # 3. 결과 확인
    print(f"\n최종 결과:")
    print(f"훈련 데이터: {result['train'].shape}")
    print(f"검증 데이터: {result['val'].shape}")
    print(f"테스트 데이터: {result['test'].shape}")
    print(f"백테스팅 데이터: {len(result['backtest_data'])}개 종목")

📁 50개 최적화 파일 로딩 중...
✅ AAVE: (1821, 58)
✅ ADA: (2879, 58)
✅ AETHWETH: (138, 58)
✅ ALGO: (2290, 58)
✅ AVAX: (1833, 58)
✅ BCH: (2879, 58)
✅ BGB: (1521, 58)
✅ BNB: (2879, 58)
✅ BTCB: (2293, 58)
✅ BTC: (3922, 58)
✅ CBBTC32994: (380, 58)
✅ CRO: (2479, 58)
✅ DAI: (2136, 58)
✅ DOGE: (2879, 58)
✅ DOT: (1864, 58)
✅ ENA: (543, 58)
✅ ETH: (2879, 58)
✅ HBAR: (2202, 58)
✅ HYPE32196: (302, 58)
✅ JITOSOL: (1058, 58)
✅ LEO: (2321, 58)
✅ LINK: (2879, 58)
✅ LTC: (3922, 58)
✅ MNT27075: (800, 58)
✅ NEAR: (1809, 58)
✅ OKB: (2342, 58)
✅ PEPE24478: (894, 58)
✅ SEI: (771, 58)
✅ SHIB: (1842, 58)
✅ SOL: (1996, 58)
✅ STETH: (1739, 58)
✅ SUI20947: (878, 58)
✅ SUSDE: (583, 58)
✅ TAO22974: (937, 58)
✅ TON11419: (1492, 58)
✅ TRX: (2879, 58)
✅ UNI7083: (1835, 58)
✅ USDC: (2546, 58)
✅ USDE29470: (585, 58)
✅ USDS33039: (373, 58)
✅ USDT: (2879, 58)
✅ WBETH: (884, 58)
✅ WBTC: (2432, 58)
✅ WEETH: (654, 58)
✅ WETH: (2812, 58)
✅ WSTETH: (1451, 58)
✅ WTRX: (1302, 58)
✅ XLM: (2879, 58)
✅ XMR: (2879, 58)
✅ XRP: (2879, 58)
🎯 로딩